In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
import itertools
from matplotlib import ticker
import matplotlib.colors as colors
from pylab import rcParams

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm1.head(2)

In [ ]:
slurm = slurm1[['UID','JobID','Account','Submittimestamp','Start','End','Eligibletimestamp','Starttimestamp','Endtimestamp','AllocNodes','AllocCPUS','TimelimitRaw', 'NNodes','Submit','State','ExitCode','QOS','QOSRAW','Partition','TRESAllocMem','TRESINMaxUsagemem','TotalCPUtime']]

In [ ]:
slurm = slurm[slurm["Starttimestamp"] != 'Unknown']
slurm = slurm[slurm["Endtimestamp"] != 'Unknown']
slurm['JobLen'] = slurm['Endtimestamp'].astype(float) - slurm['Starttimestamp'].astype(float)
slurm['Totalres'] = slurm['JobLen'] * slurm['NNodes']
slurm['JobID'] = slurm['JobID'].astype(str)

In [ ]:
discard = ["."]
waittime = slurm[~slurm.JobID.str.contains('|'.join(discard),regex = False)]
waittime['Jcount'] = 1
waittime['Waittime'] = waittime['Starttimestamp'].astype(float) - waittime['Submittimestamp'].astype(float)

In [ ]:
waittime['year'] = pd.DatetimeIndex(waittime['Submit']).year
waittime['month'] = pd.DatetimeIndex(waittime['Submit']).month
waittime['year'] = waittime['year'].apply(str)
waittime['month'] = waittime['month'].apply(str)
waittime['yearmonth'] = waittime['year'] + '-' + waittime['month']

In [ ]:
waittime.head()

In [ ]:
waittime = waittime.loc[waittime.JobLen >= 0]

In [ ]:
waittime.info()

# Wait Time Analysis

## Jobs count every year and total waittime

In [ ]:
def function2(years):
    coun = 0
    df_list = []
    df_list2 = []
    varlist = []
    for i in years:
        temp = waittime[waittime.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)
    for j in df_list:
        temp1 = j.groupby(['month','year'], as_index=False)['Waittime'].mean()
        temp1['Waittime'] = temp1['Waittime'] / 3600
        temp1['Waittime'] = temp1['Waittime'].astype(int)
        temp1['month'] = temp1['month'].apply(int)
        temp1 = temp1.sort_values('month')
        temp2 = j.Waittime.std()
        varlist.append(temp2)
        df_list2.append(temp1)
    for l in df_list2:
        count = 0
        num = l.month
        datetime_object = []
        month_name = []
        for m in num:
            datetime_object.append(datetime.strptime(str(m), "%m"))
        for d in datetime_object:
            month_name.append(d.strftime("%b"))
        fig, (ax, ax_table) = plt.subplots(nrows=2)
        ax_table.axis('off')
        ax.grid(False)
        ax.set_title("Mean Waittime every month for the year {} in hours".format(str(l.year[0])),size=18)
        width = .5 
        ax.bar(l.month, l.Waittime, width, color='g')
        fig.set_figheight(10)
        fig.set_figwidth(16)
        for p in ax.patches:
            ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=16)
        ax.set_xlabel('Months', fontsize=16)
        ax.set_ylabel('Waittime in hours', fontsize=16)
        ax.set(xticks = l.month)
        ax.set_xticklabels(month_name, fontsize=16)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")
        ax.get_yaxis().get_major_formatter().labelOnlyBase = False
        if len(varlist) != 0:
            for index,li in enumerate(varlist):
                if count == 1:
                    break
                props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
                textstr = '\n'.join((
                r'$SD= %.2f$' % (li, ),))
                ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
                verticalalignment='top', bbox=props)
                plt.subplots_adjust(left=0, bottom=0)
                count = count + 1
                varlist.pop(0)
            plt.show()
            coun = coun + 1
            if not os.path.exists('Waittime_each_Year'):
                os.makedirs('Waittime_each_Year')
            fig.savefig('Waittime_each_Year/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)
                
            

In [ ]:
years = ['2019', '2020', '2021', '2022']
function2(years)

## Number of jobs according to waittime every year

In [ ]:
def f(row):
    if row['Waittime'] <= 300:
        val = 1
    else:
        val = 0
    return val
waittime['upto5mins'] = waittime.apply(f,axis =1)
def f1(row):
    if row['Waittime'] <= 3600 and row['Waittime'] > 300:
        val = 1
    else:
        val = 0
    return val
waittime['uptoonehour'] = waittime.apply(f1,axis =1)
def f2(row):
    if row['Waittime'] <= 21600 and row['Waittime'] > 3600:
        val = 1
    else:
        val = 0
    return val
waittime['uptosix'] = waittime.apply(f2,axis =1)

def f3(row):
    if row['Waittime'] > 21600 and row['Waittime'] <= 43200:
        val = 1
    else:
        val = 0
    return val
waittime['uptotwelve'] = waittime.apply(f3,axis =1)
def f4(row):
    if row['Waittime'] > 604800:
        val = 1
    else:
        val = 0
    return val
waittime['morethanweek'] = waittime.apply(f4,axis =1)
def f5(row):
    if row['Waittime'] == 0:
        val = 1
    else:
        val = 0
    return val
waittime['nowaittime'] = waittime.apply(f5,axis =1)
def f6(row):
    if row['Waittime'] > 86400 and row['Waittime']<= 604800:
        val = 1
    else:
        val = 0
    return val
waittime['uptoweek'] = waittime.apply(f6,axis =1)
def f7(row):
    if row['Waittime'] > 43200 and row['Waittime']<= 86400:
        val = 1
    else:
        val = 0
    return val
waittime['uptoday'] = waittime.apply(f7,axis =1)

In [ ]:
def function5(years):
    coun = 0
    df_list = []
    df_list2 = []
    for i in years:
        temp = waittime[waittime.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)
    for j in df_list:
        temp1 = j.groupby(['month','year'], as_index=False).agg({'Jcount':'sum','upto5mins':'sum','uptoonehour':'sum',
                                                      'uptotwelve':'sum','uptoweek':'sum','uptoday':'sum','morethanweek':'sum','uptosix':'sum' })
        temp1['month'] = temp1['month'].apply(int)
        temp1 = temp1.sort_values('month')
        df_list2.append(temp1)
    for l in df_list2:
        count = 0
        num = l.month
        datetime_object = []
        month_name = []
        for m in num:
            datetime_object.append(datetime.strptime(str(m), "%m"))
        for d in datetime_object:
            month_name.append(d.strftime("%b"))
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to waittime duration for the year {} ".format(str(l.year[0])),size=20)
        width = .5 
        # ax.bar(l.month, l.Jcount, width, color='w', alpha = 0.8)
        ax.bar(l.month, l.upto5mins, width, color='blue',alpha = 0.5)
        ax.bar(l.month, l.uptoonehour, width,bottom = l.upto5mins, color='green',alpha = 0.5)
        ax.bar(l.month, l.uptosix, width,bottom = l.upto5mins + l.uptoonehour,  color='m',alpha = 0.5)
        ax.bar(l.month, l.uptotwelve, width,bottom = l.upto5mins + l.uptoonehour + l.uptosix,  color='brown',alpha = 0.5)
        ax.bar(l.month, l.uptoday, width,bottom = l.upto5mins + l.uptoonehour + l.uptosix + l.uptotwelve,  color='r',alpha = 0.5)
        ax.bar(l.month, l.uptoweek, width,bottom = l.upto5mins + l.uptoonehour + l.uptosix + l.uptotwelve + l.uptoday , color='yellow',alpha = 0.5)
        ax.bar(l.month, l.morethanweek, width, bottom = l.upto5mins + l.uptoonehour + l.uptosix + l.uptotwelve + l.uptoday + l.uptoweek ,color='k',alpha = 0.5)
        
        fig.set_figheight(18)
        fig.set_figwidth(16)
        # for p in ax.patches:
        #     ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=12)
        ax.set_xlabel('Months', fontsize=20)
        ax.set_ylabel('Number of jobs', fontsize=20)
        ax.set(xticks = l.month)
        ax.set_xticklabels(month_name, fontsize=18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.subplots_adjust(left=0, bottom=0)
        plt.legend(["Waittime upto 5 minutes",'Waittime from 5 to 60 minutes','Waittime from hour to 6 hours','Waittime from 6 to 12 hours','Waittime from 12 to 24 hours','Waittime from 1 day to 1 week','Waittime for more than a week' ],prop={'size': 20})
        plt.show()
        coun = coun + 1
        if not os.path.exists('Waittime_no_of_jobs'):
            os.makedirs('Waittime_no_of_jobs')
        fig.savefig('Waittime_no_of_jobs/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
years = ['2019', '2020', '2021', '2022']
function5(years)

In [ ]:
# def function6(years):
#     coun = 100
#     df_list = []
#     df_list2 = []
#     for i in years:
#         temp = waittime[waittime.yearmonth.str.contains(i,regex = False)]
#         df_list.append(temp)
#     for j in df_list:
#         temp1 = j.groupby(['month','year'], as_index=False).agg({'Jcount':'sum','upto5mins':'sum','uptoonehour':'sum',
#                                                       'uptotwelve':'sum','uptoweek':'sum','morethanweek':'sum','uptosix':'sum','uptoday':'sum' })
#         temp1['month'] = temp1['month'].apply(int)
#         temp1 = temp1.sort_values('month')
#         df_list2.append(temp1)
#     for l in df_list2:
#         count = 0
#         num = l.month
#         datetime_object = []
#         month_name = []
#         for m in num:
#             datetime_object.append(datetime.strptime(str(m), "%m"))
#         for d in datetime_object:
#             month_name.append(d.strftime("%b"))
#         fig,ax = plt.subplots()
#         ax.grid(False)
#         ax.set_title("number of jobs according to waittime duration for the year {}".format(l.year[0]),size=18)
#         width = .5 
#         ax.bar(l.month, l.Jcount, width, color='c', alpha = 0.2)
#         ax.bar(l.month, l.uptosix, width, color='g',alpha = 0.3)
#         ax.bar(l.month, l.uptoday, width, color='b',alpha = 0.3)
#         ax.set(xticks = l.month, xticklabels=month_name)
#         fig.set_figheight(10)
#         fig.set_figwidth(16)
#         for p in ax.patches:
#             ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=12)
#         ax.set_xlabel('Months', fontsize=14)
#         ax.set_ylabel('Number of jobs', fontsize=14)
#         plt.subplots_adjust(left=0, bottom=0)
#         plt.legend(["Total Jobs", "Waittime upto 6 hours",'Waittime from 6 to 24 hours'])
#         coun = coun + 1
#         if not os.path.exists('Waittime_no_of_jobs'):
#             os.makedirs('Waittime_no_of_jobs')
#         fig.savefig('Waittime_no_of_jobs/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)
        
#         plt.show()

In [ ]:
# years = ['2019', '2020', '2021', '2022']
# function6(years)

In [ ]:
# years = ['2019', '2020', '2021', '2022']
# function7(years)

## Mean waittime according to project

In [ ]:
waittime_project = waittime.groupby('Account', as_index=False).agg({'Waittime':'mean','Jcount':'sum'})
waittime_project['Account'] = waittime_project['Account'].apply(int)
waittime_project = waittime_project.sort_values('Account')

In [ ]:
waittime_project1 = waittime_project.loc[waittime_project.Account <= 40]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] =38, 10
waittime_project1['Account'] = waittime_project1['Account'].apply(str)
waittime_project1['Waittime'] = waittime_project1['Waittime'] / 3600
plots = sns.barplot(x ='Account',y = 'Waittime',data =  waittime_project1)
plots.set_title('Mean waittime according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(),'.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 0)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Account',size = 22)
plt.ylabel('Wait time in hours',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Wait_Time_project1.png')

In [ ]:
waittime_project2 = waittime_project.loc[(waittime_project.Account >40) & (waittime_project.Account <= 82)]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] =38, 10
waittime_project2['Account'] = waittime_project2['Account'].apply(str)
waittime_project2['Waittime'] = waittime_project2['Waittime'] / 3600
plots = sns.barplot(x ='Account',y = 'Waittime',data =  waittime_project2)
plots.set_title('Mean waittime according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(),'.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 0)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Account',size = 22)
plt.ylabel('Wait time in hours',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Wait_Time_project2.png')

## Job count according to project

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] =38, 10
waittime_project1['Account'] = waittime_project1['Account'].apply(str)
plots = sns.barplot(x ='Account',y = 'Jcount',data =  waittime_project1)
plots.set_title('Total jobs according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(int(bar.get_height()),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 0)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Account',size = 22)
plt.ylabel('Jobs count',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Jobs_count_project1.png')

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] =38, 10
waittime_project2['Account'] = waittime_project2['Account'].apply(str)
plots = sns.barplot(x ='Account',y = 'Jcount',data =  waittime_project2)
plots.set_title('Total jobs according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(int(bar.get_height()),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 0)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Account',size = 22)
plt.ylabel('Jobs count',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Jobs_count_project2.png')

## Wait time according to number of nodes

In [ ]:
waittime_jobsize = waittime.groupby('NNodes', as_index=False).agg({'Waittime':'mean','Jcount':'sum'})
waittime_jobsize['NNodes'] = waittime_jobsize['NNodes'].apply(int)
waittime_jobsize = waittime_jobsize.sort_values('NNodes')


In [ ]:
waittime_jobsize1 = waittime_jobsize.loc[waittime_jobsize.NNodes <= 50]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 30, 10
waittime_jobsize1['NNodes'] = waittime_jobsize1['NNodes'].apply(str)
waittime_jobsize1['Waittime'] = waittime_jobsize1['Waittime'] / 3600
plots = sns.barplot(x ='NNodes',y = 'Waittime',data =  waittime_jobsize1)
plots.set_title('Mean waittime according to different job sizes',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Job size',size = 22)
plt.ylabel('Wait time in hours',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Wait_Time_Job_Size1.png')

In [ ]:
waittime_jobsize2 = waittime_jobsize.loc[waittime_jobsize.NNodes > 50]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 30, 10
waittime_jobsize2['NNodes'] = waittime_jobsize2['NNodes'].apply(str)
waittime_jobsize2['Waittime'] = waittime_jobsize2['Waittime'] / 3600
plots = sns.barplot(x ='NNodes',y = 'Waittime',data =  waittime_jobsize2)
plots.set_title('Mean waittime according to different job sizes',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Job size',size = 22)
plt.ylabel('Wait time in hours',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Wait_Time_Job_Size2.png')

## Waittime according to QOS

In [ ]:
waittime_QOS = waittime.groupby('QOS', as_index=False).agg({'Waittime':'mean','Jcount':'sum'})
waittime_QOS = waittime_QOS.loc[(waittime_QOS.QOS == 'cont') | (waittime_QOS.QOS == 'lowcont') | (waittime_QOS.QOS == 'nocont')]

In [ ]:
waittime_QOS

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
waittime_QOS['QOS'] = waittime_QOS['QOS'].apply(str)
waittime_QOS['Waittime'] = waittime_QOS['Waittime'] / 3600
plots = sns.barplot(x ='QOS',y = 'Waittime',data =  waittime_QOS)
plots.set_title('Mean wait time according to different QOS',size = 18)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=16, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('QoS',size = 18)
plt.ylabel('Wait time in hours',size = 18)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Wait_Time_QOS11.png')

## Waittime according to partition

In [ ]:
waittime_Partition = waittime.groupby('Partition', as_index=False).agg({'Waittime':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
waittime_Partition['Partition'] = waittime_Partition['Partition'].apply(str)
waittime_Partition['Waittime'] = waittime_Partition['Waittime'] / 3600
plots = sns.barplot(x ='Partition',y = 'Waittime',data =  waittime_Partition)
plots.set_title('Mean wait time according to different Partitions',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Partition',size = 22)
plt.ylabel('Wait time in hours',size = 22)
if not os.path.exists('Wait_Time_resources'):
    os.makedirs('Wait_Time_resources')
plt.savefig('Wait_Time_resources/Wait_Time_Partition 1.png')